In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train=pd.read_csv("/content/train_MpHjUjU.csv")

In [4]:
test=pd.read_csv("/content/test_hXY9mYw.csv")

In [5]:
train.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,-665480,2
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1


In [11]:
#as we can see rows which have lastworkingdate not null are the employees who have already left the organization
# so, i want to compare their feature values with other employees,and i will form clusters with the people having the same similar data  

So, i want to analyse the data of employees who already left the organisation 

In [6]:
leavelist=train[~(train["LastWorkingDate"].isnull())]

In [7]:
leavelist.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2
9,2017-04-01,4,43,Male,C13,Master,65603,2016-12-07,2017-04-27,2,2,0,1
12,2016-03-01,5,29,Male,C9,College,46368,2016-01-09,2016-03-07,1,1,0,1
20,2017-11-01,8,34,Male,C2,College,70656,2017-09-19,2017-11-15,3,3,0,1
27,2016-12-01,12,35,Male,C23,Master,28116,2016-06-29,2016-12-21,1,1,0,1


In [ ]:
# personal details like gender,city,education_level would have less effect on attrition,so i'm not considering those features 

In [8]:
leavelist.shape

(1616, 13)

In [ ]:
#we have 1616 employees data who have already left the organisation
#so out of 1616 employees,people with same designation have left the organization more
#and people with quarterly rating 1 have left the organisation more

In [9]:
leavelist["Quarterly Rating"].value_counts()

1    1432
2     146
3      28
4      10
Name: Quarterly Rating, dtype: int64

In [10]:
leavelist[leavelist["Joining Designation"]==leavelist["Designation"]].shape

(1395, 13)

In [12]:
train=pd.get_dummies(train,columns=["Joining Designation","Designation"],drop_first=True)

In [13]:
train.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Total Business Value,Quarterly Rating,Joining Designation_2,Joining Designation_3,Joining Designation_4,Joining Designation_5,Designation_2,Designation_3,Designation_4,Designation_5
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,2381060,2,0,0,0,0,0,0,0,0
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,-665480,2,0,0,0,0,0,0,0,0
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,0,2,0,0,0,0,0,0,0,0
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,0,1,1,0,0,0,1,0,0,0
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,0,1,1,0,0,0,1,0,0,0


In [14]:
train1=train[["Joining Designation_2","Joining Designation_3","Joining Designation_4","Joining Designation_5","Designation_3","Designation_4","Designation_5","Quarterly Rating"]]

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
scalar=StandardScaler()

In [17]:
y=train[["Salary","Total Business Value"]]

In [20]:
traino=scalar.fit_transform(y.to_numpy())

In [21]:
traino

array([[-0.26735795,  1.60367428],
       [-0.26735795, -1.09648237],
       [-0.26735795, -0.50666564],
       ...,
       [ 0.14886519, -0.50666564],
       [ 0.14886519, -0.32903286],
       [ 0.14886519, -0.14196981]])

In [22]:
traino=pd.DataFrame(traino,columns=["scaled_salary","scaled_bussiness_value"])

In [23]:
train=pd.concat([train,traino],axis=1)

In [24]:
train.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Total Business Value,Quarterly Rating,Joining Designation_2,Joining Designation_3,Joining Designation_4,Joining Designation_5,Designation_2,Designation_3,Designation_4,Designation_5,scaled_salary,scaled_bussiness_value
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,2381060,2,0,0,0,0,0,0,0,0,-0.267358,1.603674
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,-665480,2,0,0,0,0,0,0,0,0,-0.267358,-1.096482
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,0,2,0,0,0,0,0,0,0,0,-0.267358,-0.506666
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,0,1,1,0,0,0,1,0,0,0,0.044122,-0.506666
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,0,1,1,0,0,0,1,0,0,0,0.044122,-0.506666


In [25]:
train1=train[["Joining Designation_2","Joining Designation_3","Joining Designation_4","Joining Designation_5","Designation_3","Designation_4","Designation_5","Quarterly Rating","scaled_salary","scaled_bussiness_value"]]

In [26]:
from sklearn.cluster import KMeans

In [27]:
kmeans=KMeans(n_clusters=2)

In [28]:
kmeans.fit(train1)

KMeans(n_clusters=2)

In [29]:
n=pd.DataFrame(kmeans.labels_,columns=["clusters"])

In [30]:
train=pd.concat([train,n],axis=1)

In [31]:
train.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Total Business Value,Quarterly Rating,Joining Designation_2,Joining Designation_3,Joining Designation_4,Joining Designation_5,Designation_2,Designation_3,Designation_4,Designation_5,scaled_salary,scaled_bussiness_value,clusters
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,2381060,2,0,0,0,0,0,0,0,0,-0.267358,1.603674,1
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,-665480,2,0,0,0,0,0,0,0,0,-0.267358,-1.096482,0
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,0,2,0,0,0,0,0,0,0,0,-0.267358,-0.506666,0
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,0,1,1,0,0,0,1,0,0,0,0.044122,-0.506666,0
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,0,1,1,0,0,0,1,0,0,0,0.044122,-0.506666,0


In [36]:
cluster1=train[train["clusters"]==0]

In [37]:
cluster1.shape

(11934, 22)

In [39]:
cluster1[~(cluster1["LastWorkingDate"].isnull())].shape

(1564, 22)

In [ ]:
# so,most of the employees who have left the organisation fall in cluster 0
#this is the cluster with employees similar to the employees left the oraganisation

In [ ]:
# as the problem statement states that the people who leave the organisation 
#represent as 1,otherwise 0
#we are making the cluster 0 to 1 and cluster 1 to 0

In [40]:
g={0:1,1:0}

In [41]:
train["target"]=train["clusters"].map(g)

In [ ]:
#as it has so many duplicates,we are droping the duplicate emp_ids

In [42]:
train2=train.drop_duplicates(subset="Emp_ID",keep="last")

In [43]:
train2.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Total Business Value,Quarterly Rating,Joining Designation_2,Joining Designation_3,Joining Designation_4,Joining Designation_5,Designation_2,Designation_3,Designation_4,Designation_5,scaled_salary,scaled_bussiness_value,clusters,target
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,0,2,0,0,0,0,0,0,0,0,-0.267358,-0.506666,0,1
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,0,1,1,0,0,0,1,0,0,0,0.044122,-0.506666,0,1
9,2017-04-01,4,43,Male,C13,Master,65603,2016-12-07,2017-04-27,0,1,1,0,0,0,1,0,0,0,-0.001586,-0.506666,0,1
12,2016-03-01,5,29,Male,C9,College,46368,2016-01-09,2016-03-07,0,1,0,0,0,0,0,0,0,0,-0.623802,-0.506666,0,1
17,2017-12-01,6,31,Female,C11,Bachelor,78728,2017-07-31,NaN,0,2,0,1,0,0,0,1,0,0,0.422983,-0.506666,0,1


In [44]:
train2.index=train2.Emp_ID

In [45]:
train2.head()

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Total Business Value,Quarterly Rating,Joining Designation_2,Joining Designation_3,Joining Designation_4,Joining Designation_5,Designation_2,Designation_3,Designation_4,Designation_5,scaled_salary,scaled_bussiness_value,clusters,target
Emp_ID,,,,,,,,,,,,,,,,,,,,,,,
1,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,0,2,0,0,0,0,0,0,0,0,-0.267358,-0.506666,0,1
2,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,0,1,1,0,0,0,1,0,0,0,0.044122,-0.506666,0,1
4,2017-04-01,4,43,Male,C13,Master,65603,2016-12-07,2017-04-27,0,1,1,0,0,0,1,0,0,0,-0.001586,-0.506666,0,1
5,2016-03-01,5,29,Male,C9,College,46368,2016-01-09,2016-03-07,0,1,0,0,0,0,0,0,0,0,-0.623802,-0.506666,0,1
6,2017-12-01,6,31,Female,C11,Bachelor,78728,2017-07-31,NaN,0,2,0,1,0,0,0,1,0,0,0.422983,-0.506666,0,1


In [47]:
subfile=pd.read_csv("sample_submission_znWiLZ4.csv")

In [49]:
subfile.Target=subfile["Emp_ID"].map(lambda x : train2.loc[x]["target"])


In [50]:
subfile

,Emp_ID,Target
0,394,0
1,173,0
2,1090,0
3,840,0
4,308,0
...,...,...
736,2134,1
737,2255,0
738,448,0
739,1644,1


In [51]:
subfile.to_csv("submitfd.csv")